In [ ]:
import pickle
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
from tqdm.notebook import tqdm


from sklearn.preprocessing import LabelEncoder
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import pylab
import pickle
import numpy as np
import time
import itertools
from tqdm.notebook import tqdm
from multiprocessing import Pool
import matplotlib.cm as cm
import scipy
import community as community_louvain
import seaborn as sns
import os
from collections import defaultdict
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
from sklearn.metrics import accuracy_score,classification_report
import keras

import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.2.1

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.2.1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.2.1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

In [ ]:
from stellargraph import StellarGraph,StellarDiGraph

In [ ]:
from stellargraph import datasets, utils
from tensorflow.keras import callbacks, optimizers, losses, metrics, regularizers, Model
import numpy as np
import pandas as pd

from stellargraph.mapper import KGTripleGenerator
from stellargraph.layer import ComplEx,DistMult

from IPython.display import HTML

In [ ]:
cd drive/MyDrive/gnn

/content/drive/MyDrive/gnn


The detailed data can be downloaded here: http://snap.stanford.edu/conflict/conflict_data.zip

In [ ]:
DATA_DIR = './prediction'

In [ ]:
# loading handcrafted features
meta_features = {}
meta_labels = {}
with open(DATA_DIR+"/detailed_data/handcrafted_features.tsv") as fp:
    for line in fp:
        info = line.split()
        meta_features[info[0]] = np.array(list(map(float, info[-1].split(","))))
        meta_labels[info[0]] = 1 if info[1] == "burst" else 0

In [ ]:
# loading the user, source, and target community embeddings for all examples
with open(DATA_DIR + "/detailed_data/full_ids.txt") as fp:
    ids = {id.strip():i for i, id in enumerate(fp.readlines())}


In [ ]:
# loading the post embeddings from the LSTM 
lstm_ids = pickle.load(open(DATA_DIR + "/detailed_data/lstm_embeds-ids.pkl",'rb'))
lstm_ids = {id:i for i, id in enumerate(lstm_ids)}

In [ ]:
full_embeds_array = np.load('./prediction/detailed_data/full_embeds.npy')
full_embeds=pd.DataFrame(full_embeds_array)

In [ ]:
lstm_embeds_array = np.load('./prediction/detailed_data/lstm_embeds.npy')
lstm_embeds=pd.DataFrame(lstm_embeds_array)

In [ ]:

# loading preprocessed lstm data to ensure identical train/val/test splits
train_data = pickle.load(open(DATA_DIR + "/preprocessed_train_data.pkl",'rb'))
val_data = pickle.load(open(DATA_DIR + "/preprocessed_val_data.pkl",'rb'))
test_data = pickle.load(open(DATA_DIR + "/preprocessed_test_data.pkl",'rb'))

In [ ]:
# flattening the preprocessed LSTM data (no need for minibatching here....)
def flatten(data):
    ids, text, users, subreddits, lengths,sfs, labels = [], [], [], [], [], [], []

    for batch in data:
        bids, btext, busers, bsubreddits, blengths, bsfs, blabels = batch
        ids.extend([x.decode('utf-8') for x in bids])
        text.extend(btext.numpy().tolist())
        users.extend(busers.numpy().tolist())
        subreddits.extend(bsubreddits.numpy().tolist())
        lengths.extend(blengths)
        labels.extend(blabels)
        sfs.extend(bsfs)
    return (ids, text, users, subreddits, lengths, labels)
flat_train_data = flatten(train_data)
flat_val_data = flatten(val_data)
flat_test_data = flatten(test_data)
print(len(flat_train_data[0]))

93696


In [ ]:
lst_ids_parse = [(x.decode('utf-8'),y) for x,y in list(lstm_ids.items())]

In [ ]:
lstm_ids = dict(lst_ids_parse)

In [ ]:
df = pd.read_csv('soc-redditHyperlinks-body.tsv',sep='\t')


In [ ]:
df['post_id_cropped'] = df['POST_ID'].apply(lambda x: x[:-1] if len(x) == 7 else x)

In [ ]:
df_post_idx = df.set_index('post_id_cropped')
df_post_idx = df_post_idx.drop('POST_ID', 1)

In [ ]:
###DISTMULT

In [ ]:
df = pd.read_csv('soc-redditHyperlinks-body.tsv',sep='\t')

In [ ]:
nodes = df['SOURCE_SUBREDDIT'].unique().tolist()
nodes=nodes.extend(df['TARGET_SUBREDDIT'].unique().tolist())

In [ ]:
reddit_edges = pd.DataFrame(
    {"source": df['SOURCE_SUBREDDIT'], "target": df['TARGET_SUBREDDIT']}
)
print(reddit_edges)

                     source             target
0           leagueoflegends    teamredditteams
1                theredlion             soccer
2              inlandempire             bikela
3                       nfl                cfb
4                playmygame            gamedev
...                     ...                ...
286556           negareddit      debatefascism
286557          mildlynomil          justnomil
286558               mmorpg  blackdesertonline
286559  electricskateboards          askreddit
286560                mgtow    dataisbeautiful

[286561 rows x 2 columns]


In [ ]:
df['LINK_SENTIMENT'][df['LINK_SENTIMENT']==-1]="neg"
df['LINK_SENTIMENT'][df['LINK_SENTIMENT']==1]="pos"

neglen=df['LINK_SENTIMENT'][df['LINK_SENTIMENT']=="neg"].shape[0]
poslen=df['LINK_SENTIMENT'][df['LINK_SENTIMENT']=="pos"].shape[0]


reddit_edges = reddit_edges.assign(
    label=df['LINK_SENTIMENT']
)
print(reddit_edges)
print(neglen,poslen)

total=neglen+poslen
weight_for_0 = (1 / neglen)*(total)/2.0 
weight_for_1 = (1 / poslen)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

                     source             target label
0           leagueoflegends    teamredditteams   pos
1                theredlion             soccer   neg
2              inlandempire             bikela   pos
3                       nfl                cfb   pos
4                playmygame            gamedev   pos
...                     ...                ...   ...
286556           negareddit      debatefascism   pos
286557          mildlynomil          justnomil   pos
286558               mmorpg  blackdesertonline   pos
286559  electricskateboards          askreddit   pos
286560                mgtow    dataisbeautiful   pos

[286561 rows x 3 columns]
21070 265491
Weight for class 0: 6.80
Weight for class 1: 0.54


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
reddit_edges_stellar = StellarDiGraph(
    edges=reddit_edges, edge_type_column="label"
)


In [ ]:
from sklearn import model_selection

reddit_train, reddit_test = model_selection.train_test_split(
    reddit_edges, train_size=0.20, random_state=123
)
print(reddit_train)

                     source                target label
254242              askmath             learnmath   pos
137301                 sb4b    imaginarydialogues   pos
200188                 vive                 kodon   pos
89634   mindcrackdiscussion             mindcrack   pos
225494            smashbros                 ssbpm   pos
...                     ...                   ...   ...
192476              askcops         legaladviceuk   neg
17730            futurology  thisisthewayitwillbe   pos
28030                 drugs             askreddit   pos
277869           parahumans          mylittlepony   pos
249342                  nfl       tennesseetitans   pos

[57312 rows x 3 columns]


In [ ]:
"""train_reddit_edges_stellar = StellarGraph(
    edges=reddit_train, edge_type_column="label"
)
print(train_reddit_edges_stellar.info())"""

'train_reddit_edges_stellar = StellarGraph(\n    edges=reddit_train, edge_type_column="label"\n)\nprint(train_reddit_edges_stellar.info())'

In [ ]:
"""test_reddit_edges_stellar = StellarGraph(
    edges=reddit_test, edge_type_column="label"
)"""


'test_reddit_edges_stellar = StellarGraph(\n    edges=reddit_test, edge_type_column="label"\n)'

In [ ]:
epochs = 50
embedding_dimension = 300
negative_samples = 10

wn18_gen = KGTripleGenerator(
    reddit_edges_stellar, batch_size=len(reddit_train) // 100  # ~100 batches per epoch
)

wn18_complex = DistMult(
    wn18_gen,
    embedding_dimension=embedding_dimension,
    embeddings_regularizer=regularizers.l2(1e-7),
)



wn18_inp, wn18_out = wn18_complex.in_out_tensors()



node_embs_complex,edge_embs_complex=wn18_complex.embeddings()
print(node_embs_complex)

embeddingTable_compex = {}

for x in range(node_embs_complex.shape[0]):
    #print(reddit_edges_stellar.nodes()[x])
    embeddingTable_compex[reddit_edges_stellar.nodes()[x]] = node_embs_complex[x]



distmult_embdf=pd.DataFrame.from_dict(embeddingTable_compex).T
#print(distmult_embdf)


[[ 0.04968078  0.03343967 -0.0060774  ... -0.02498448  0.02389926
   0.03230396]
 [-0.01420367  0.03767251 -0.0139218  ... -0.02904452  0.00026305
  -0.00976703]
 [-0.01683909  0.02301161 -0.04011309 ... -0.00927961  0.01154182
   0.04704023]
 ...
 [ 0.02488888 -0.01880057 -0.02210356 ... -0.00872957  0.04767818
   0.04802516]
 [-0.02115102 -0.00122106  0.02790412 ... -0.00189884  0.01677041
  -0.01933175]
 [ 0.04618433  0.02606359  0.01553662 ... -0.04082925  0.02861075
   0.00217636]]


In [ ]:
only_user_emb=full_embeds.iloc[:,:300]

In [ ]:
def get_gnn_embeddings2(ids_list,subreddit_embdf):
    gnn_embs_source = {}
    gnn_embs_target = {}

    not_found_ids = list()
    for x in tqdm(ids_list):

        chunk = df_post_idx.loc[x]
        if not isinstance(chunk,pd.core.series.Series):
            source = chunk.SOURCE_SUBREDDIT.iloc[0]
            target = chunk.TARGET_SUBREDDIT.iloc[0]
        else:
            source = chunk.SOURCE_SUBREDDIT
            target = chunk.TARGET_SUBREDDIT

        try:
            gnn_embs_source[x]=subreddit_embdf.loc[source].values.tolist()
            gnn_embs_target[x]=subreddit_embdf.loc[target].values.tolist()
        except IndexError:
            print(x)
            not_found_ids.append(x)


      
    return gnn_embs_source,gnn_embs_target,not_found_ids

In [ ]:
###for distmult-embeddings

In [ ]:
train_distmult_embs_source,train_distmult_embs_target,unfound_ids_train = get_gnn_embeddings2(flat_train_data[0],distmult_embdf)

In [ ]:
val_distmult_embs_source,val_distmult_embs_target,unfound_ids_val = get_gnn_embeddings2(flat_val_data[0],distmult_embdf)

In [ ]:
test_distmult_embs_source,test_distmult_embs_target,unfound_ids_test = get_gnn_embeddings2(flat_test_data[0],distmult_embdf)

In [ ]:
dftrain_distmult_embs_source=pd.DataFrame.from_dict(train_distmult_embs_source).T
dfval_distmult_embs_source=pd.DataFrame.from_dict(val_distmult_embs_source).T
dftest_distmult_embs_source=pd.DataFrame.from_dict(test_distmult_embs_source).T

dftrain_distmult_embs_target=pd.DataFrame.from_dict(train_distmult_embs_target).T
dfval_distmult_embs_target=pd.DataFrame.from_dict(val_distmult_embs_target).T
dftest_distmult_embs_target=pd.DataFrame.from_dict(test_distmult_embs_target).T

In [ ]:
del train_distmult_embs_source,train_distmult_embs_target,val_distmult_embs_source,val_distmult_embs_target,test_distmult_embs_source,test_distmult_embs_target

In [ ]:
train_XXX = np.stack([np.concatenate([meta_features[i], full_embeds.iloc[ids[i]], dftrain_distmult_embs_source.loc[i],dftrain_distmult_embs_target.loc[i]]) for i in flat_train_data[0] if (i not in unfound_ids_train)])
val_XXX =  np.stack([np.concatenate([meta_features[i], full_embeds.iloc[ids[i]], dfval_distmult_embs_source.loc[i],dfval_distmult_embs_target.loc[i]]) for i in flat_val_data[0] if (i in meta_features) and (i not in unfound_ids_val)])
test_XXX =  np.stack([np.concatenate([meta_features[i],full_embeds.iloc[ids[i]], dftest_distmult_embs_source.loc[i],dftest_distmult_embs_target.loc[i]]) for i in flat_test_data[0] if (i in meta_features) and (i not in unfound_ids_test)])

train_YYY = np.stack([meta_labels[i] for i in flat_train_data[0] if (i in meta_features) and (i not in unfound_ids_train)])
val_YYY =  np.stack([meta_labels[i] for i in flat_val_data[0] if (i in meta_features) and (i not in unfound_ids_val)])
test_YYY =  np.stack([meta_labels[i] for i in flat_test_data[0] if (i in meta_features) and (i not in unfound_ids_test)])

KeyboardInterrupt: ignored

In [ ]:
# First we run the Random Forest with only the metadata/handcrafted features...
ensemble_mod = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=0)
# note that the first 263 features are the handcrafted ones... 
ensemble_mod.fit(train_XXX[:, :], train_YYY)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
###META+ FULL_EMBEDS+ SOURCE-TARGET DISTMULT EMBED.

In [ ]:
roc_auc_score(val_YYY, ensemble_mod.predict_proba(val_XXX[:, :])[:,1])

0.7545013219833208

In [ ]:
train_XXX2 = np.stack([np.concatenate([meta_features[i], only_user_emb.iloc[ids[i]], dftrain_distmult_embs_source.loc[i],dftrain_distmult_embs_target.loc[i]]) for i in flat_train_data[0] if (i not in unfound_ids_train)])
val_XXX2 =  np.stack([np.concatenate([meta_features[i], only_user_emb.iloc[ids[i]], dfval_distmult_embs_source.loc[i],dfval_distmult_embs_target.loc[i]]) for i in flat_val_data[0] if (i in meta_features) and (i not in unfound_ids_val)])
test_XXX2 =  np.stack([np.concatenate([meta_features[i],only_user_emb.iloc[ids[i]], dftest_distmult_embs_source.loc[i],dftest_distmult_embs_target.loc[i]]) for i in flat_test_data[0] if (i in meta_features) and (i not in unfound_ids_test)])

train_YYY2 = np.stack([meta_labels[i] for i in flat_train_data[0] if (i in meta_features) and (i not in unfound_ids_train)])
val_YYY2 =  np.stack([meta_labels[i] for i in flat_val_data[0] if (i in meta_features) and (i not in unfound_ids_val)])
test_YYY2 =  np.stack([meta_labels[i] for i in flat_test_data[0] if (i in meta_features) and (i not in unfound_ids_test)])

In [ ]:
# First we run the Random Forest with only the metadata/handcrafted features...
ensemble_mod = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=0)
# note that the first 263 features are the handcrafted ones... 
ensemble_mod.fit(train_XXX2[:, :], train_YYY2)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
###META+ USER EMBEDS FROM ORIG PAPER + SOURCE-TARGET DISTMULT EMBED.

In [ ]:
roc_auc_score(val_YYY2, ensemble_mod.predict_proba(val_XXX2[:, :])[:,1])

0.7330564279544989

In [ ]:
train_XXX3 = np.stack([np.concatenate([ only_user_emb.iloc[ids[i]], dftrain_distmult_embs_source.loc[i],dftrain_distmult_embs_target.loc[i]]) for i in flat_train_data[0] if (i not in unfound_ids_train)])
val_XXX3 =  np.stack([np.concatenate([only_user_emb.iloc[ids[i]], dfval_distmult_embs_source.loc[i],dfval_distmult_embs_target.loc[i]]) for i in flat_val_data[0] if (i in meta_features) and (i not in unfound_ids_val)])
test_XXX3 =  np.stack([np.concatenate([only_user_emb.iloc[ids[i]], dftest_distmult_embs_source.loc[i],dftest_distmult_embs_target.loc[i]]) for i in flat_test_data[0] if (i in meta_features) and (i not in unfound_ids_test)])

train_YYY3 = np.stack([meta_labels[i] for i in flat_train_data[0] if (i in meta_features) and (i not in unfound_ids_train)])
val_YYY3 =  np.stack([meta_labels[i] for i in flat_val_data[0] if (i in meta_features) and (i not in unfound_ids_val)])
test_YYY3 =  np.stack([meta_labels[i] for i in flat_test_data[0] if (i in meta_features) and (i not in unfound_ids_test)])

In [ ]:
# First we run the Random Forest with only the metadata/handcrafted features...
ensemble_mod = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=0)
# note that the first 263 features are the handcrafted ones... 
ensemble_mod.fit(train_XXX3[:, :], train_YYY3)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
### USER EMBEDS FROM ORIG PAPER + SOURCE-TARGET DISTMULT EMBED.

In [ ]:
roc_auc_score(val_YYY3, ensemble_mod.predict_proba(val_XXX3[:, :])[:,1])

0.7002967140710163

In [ ]:
train_XXX4 = np.stack([np.concatenate([ dftrain_distmult_embs_source.loc[i],dftrain_distmult_embs_target.loc[i]]) for i in flat_train_data[0] if (i not in unfound_ids_train)])
val_XXX4 =  np.stack([np.concatenate([ dfval_distmult_embs_source.loc[i],dfval_distmult_embs_target.loc[i]]) for i in flat_val_data[0] if (i in meta_features) and (i not in unfound_ids_val)])
test_XXX4 =  np.stack([np.concatenate([dftest_distmult_embs_source.loc[i],dftest_distmult_embs_target.loc[i]]) for i in flat_test_data[0] if (i in meta_features) and (i not in unfound_ids_test)])

train_YYY4 = np.stack([meta_labels[i] for i in flat_train_data[0] if (i in meta_features) and (i not in unfound_ids_train)])
val_YYY4 =  np.stack([meta_labels[i] for i in flat_val_data[0] if (i in meta_features) and (i not in unfound_ids_val)])
test_YYY4 =  np.stack([meta_labels[i] for i in flat_test_data[0] if (i in meta_features) and (i not in unfound_ids_test)])

In [ ]:
# First we run the Random Forest with only the metadata/handcrafted features...
ensemble_mod = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=0)
# note that the first 263 features are the handcrafted ones... 
ensemble_mod.fit(train_XXX4[:, :], train_YYY4)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
###  SOURCE-TARGET DISTMULT EMBED.

In [ ]:
roc_auc_score(val_YYY4, ensemble_mod.predict_proba(val_XXX4[:, :])[:,1])

0.6740767246995512

In [ ]:
####RUN THE CELLS UNTIL HERE#######

## Level 1 Datasets
Dataset-1 D<sub>1,1</sub> = Only Meta-Features

Dataset-2 D<sub>1,2</sub> = All embeddings

Dataset-3 D<sub>1,3</sub> = LSTM-Embeddings

Dataset-4 D<sub>1,4</sub> = Node2Vec-Embeddings

## Level 2 Datasets
Dataset-6 D<sub>2,3</sub> = All embeddings + LSTM-Embeddings

Dataset-7 D<sub>2,4</sub> = All embeddings + Node2Vec-Embeddings

## Level 3 Datasets
Dataset-8 D<sub>3,3</sub> = All embeddings + Meta-Features + LSTM-Embeddings

Dataset-8 D<sub>3,4</sub> = All embeddings + Meta-Features + Node2Vec-Embeddings

## Complete
Dataset-9 D<sub>4,3</sub> = All embeddings + Meta-Features + Node2Vec-Embeddings + LSTM-Embeddings


| Dataset | ROC_AUC | Contains Graph Embedding |
|---|---|---|
|Level 1|||
|Meta-Features| 0.6800 | No |
|All embeddings| **0.7450** | No |
|LSTM-Embeddings| 0.4926 | No |
|Node2Vec-Embeddings| 0.7170 | Yes |
|Level 2|||
|All embeddings + LSTM-Embeddings| 0.7440 | No |
|All embeddings + Node2Vec-Embeddings| 0.7450 | Yes |
|Meta-Features + Node2Vec-Embeddings| **0.7523** | Yes |
|Level 3|||
All embeddings + Meta-Features + LSTM-Embeddings| 0.7501 | No |
|All embeddings + Meta-Features + Node2Vec-Embeddings| **0.7598** | Yes |
|Level 4|||
|All available datasets| 0.7499 | No |